# Convert a TensorFlow Object Detection Model to OpenVINO™

[TensorFlow](https://www.tensorflow.org/), or TF for short, is an open-source framework for machine learning.

The [TensorFlow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) is an open-source computer vision framework built on top of TensorFlow. It is used for building object detection and image segmentation models that can localize multiple objects in the same image. TensorFlow Object Detection API supports various architectures and models, which can be found and downloaded from the [TensorFlow Hub](https://tfhub.dev/tensorflow/collections/object_detection/1).

This tutorial shows how to convert a TensorFlow [Faster R-CNN with Resnet-50 V1](https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1) object detection model to OpenVINO [Intermediate Representation](https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_IR_and_opsets.html) (OpenVINO IR) format, using [Model Optimizer](https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html). After creating the OpenVINO IR, load the model in [OpenVINO Runtime](https://docs.openvino.ai/nightly/openvino_docs_OV_UG_OV_Runtime_User_Guide.html) and do inference with a sample image.

## Prerequisites

Install required packages:

In [ ]:
!pip install -q "openvino-dev>=2023.0.0" "numpy>=1.21.0" "opencv-python" "matplotlib>=3.4,<3.5.3"

The notebook uses utility functions.
The cell below will download the `notebook_utils` Python module from GitHub.

In [ ]:
# Fetch the notebook utils script from the openvino_notebooks repo
import urllib.request

urllib.request.urlretrieve(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py",
    filename="notebook_utils.py",
)

## Imports

In [ ]:
# Standard python modules
from pathlib import Path

# External modules and dependencies
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Notebook utils module
from notebook_utils import download_file

# OpenVINO modules
from openvino.runtime import Core, serialize
from openvino.tools import mo

## Settings

Define model related variables and create corresponding directories:

In [ ]:
# Create directories for models files
model_dir = Path("model")
model_dir.mkdir(exist_ok=True)

# Create directory for TensorFlow model
tf_model_dir = model_dir / "tf"
tf_model_dir.mkdir(exist_ok=True)

# Create directory for OpenVINO IR model
ir_model_dir = model_dir / "ir"
ir_model_dir.mkdir(exist_ok=True)

model_name = "faster_rcnn_resnet50_v1_640x640"

openvino_ir_path = ir_model_dir / f"{model_name}.xml"

tf_model_url = "https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1?tf-hub-format=compressed"

tf_model_archive_filename = f"{model_name}.tar.gz"

## Download Model from TensorFlow Hub

Download archive with TensorFlow Object Detection model ([faster_rcnn_resnet50_v1_640x640](https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1)) from TensorFlow Hub:

In [ ]:
download_file(
    url=tf_model_url,
    filename=tf_model_archive_filename,
    directory=tf_model_dir
)

Extract TensorFlow Object Detection model from the downloaded archive:

In [ ]:
import tarfile

with tarfile.open(tf_model_dir / tf_model_archive_filename) as file:
    file.extractall(path=tf_model_dir)

## Convert Model to OpenVINO IR

OpenVINO Model Optimizer Python API can be used to convert the TensorFlow model to OpenVINO IR. 

`mo.convert_model` function accept path to TensorFlow model and returns OpenVINO Model class instance which represents this model.
Also we need to provide model input shape (`input_shape`) that is described at [model overview page on TensorFlow Hub](https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1). 
Optionally, we can apply compression to FP16 model weigths using `compress_to_fp16=True` option and integrate preprocessing using this approach.

The converted model is ready to load on a device using `compile_model` or saved on disk using the `serialize` function to reduce loading time when the model is run in the future. 

See the [Model Optimizer Developer Guide](https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html) for more information about Model Optimizer and TensorFlow [models suport](https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_TensorFlow.html).

In [ ]:
ov_model = mo.convert_model(
    saved_model_dir=tf_model_dir,
    input_shape=[[1, 255, 255, 3]]
)

# Save converted OpenVINO IR model to the corresponding directory
serialize(ov_model, openvino_ir_path)

## Test Inference on the Converted Model

### Load the Model

In [ ]:
ie = Core()
openvino_ir_model = ie.read_model(openvino_ir_path)
compiled_model = ie.compile_model(model=openvino_ir_model, device_name="CPU")

### Get Model Information

Faster R-CNN with Resnet-50 V1 object detection model has one input - a three-channel image of variable size. The input tensor shape is `[1, height, width, 3]` with values in `[0, 255]`.

Model output dictionary contains several tensors:
- `num_detections` - the number of detections in `[N]` format.
- `detection_boxes` - bounding box coordinates for all `N` detections in `[ymin, xmin, ymax, xmax]` format.
- `detection_classes` - `N` detection class indexes size from the label file.
- `detection_scores` - `N` detection scores (confidence) for each detected class.
- `raw_detection_boxes` - decoded detection boxes without Non-Max suppression.
- `raw_detection_scores` - class score logits for raw detection boxes.
- `detection_anchor_indices` - the anchor indices of the detections after NMS.
- `detection_multiclass_scores` - class score distribution (including background) for detection boxes in the image including background class.

In this tutorial we will mostly use `detection_boxes`, `detection_classes`, `detection_scores` tensors. It is important to mention, that values of these tensors correspond to each other and are ordered by the highest detection score: the first detection box corresponds to the first detection class and to the first (and highest) detection score.

See the [model overview page on TensorFlow Hub](https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1) for more information about model inputs, outputs and their formats.

In [ ]:
model_inputs = compiled_model.inputs
model_input = compiled_model.input(0)
model_outputs = compiled_model.outputs

print("Model inputs count:", len(model_inputs))
print("Model input:", model_input)

print("Model outputs count:", len(model_outputs))
print("Model outputs:")
for output in model_outputs:
    print("  ", output)

### Get an Image for Test Inference

Load and save an image:

In [ ]:
image_path = Path("./data/coco_bike.jpg")

download_file(
    url="https://storage.openvinotoolkit.org/repositories/openvino_notebooks/data/data/image/coco_bike.jpg",
    filename=image_path.name,
    directory=image_path.parent,
)

Read the image, resize and convert it to the input shape of the network:

In [ ]:
# Read the image
image = cv2.imread(filename=str(image_path))

# The network expects images in RGB format
image = cv2.cvtColor(image, code=cv2.COLOR_BGR2RGB)

# Resize the image to the network input shape
resized_image = cv2.resize(src=image, dsize=(255, 255))

# Transpose the image to the network input shape
network_input_image = np.expand_dims(resized_image, 0)

# Show the image
plt.imshow(image)

### Perform Inference

In [ ]:
inference_result = compiled_model(network_input_image)

After model inference on the test image, object detection data can be extracted from the result.
For further model result visualization `detection_boxes`, `detection_classes` and `detection_scores` outputs will be used.

In [ ]:
_, detection_boxes, detection_classes, _, detection_scores, num_detections, _, _ = model_outputs

image_detection_boxes = inference_result[detection_boxes]
print("image_detection_boxes:", image_detection_boxes)

image_detection_classes = inference_result[detection_classes]
print("image_detection_classes:", image_detection_classes)

image_detection_scores = inference_result[detection_scores]
print("image_detection_scores:", image_detection_scores)

image_num_detections = inference_result[num_detections]
print("image_detections_num:", image_num_detections)

# Alternatively, inference result data can be extracted by model output name with `.get()` method
assert (inference_result[detection_boxes] == inference_result.get("detection_boxes")).all(), "extracted inference result data should be equal"

### Inference Result Visualization

Define utility functions to visualize the inference results

In [ ]:
import random
from typing import Optional


def add_detection_box(box: np.ndarray, image: np.ndarray, label: Optional[str] = None) -> np.ndarray:
    """
    Helper function for adding single bounding box to the image

    Parameters
    ----------
    box : np.ndarray
        Bounding box coordinates in format [ymin, xmin, ymax, xmax]
    image : np.ndarray
        The image to which detection box is added
    label : str, optional
        Detection box label string, if not provided will not be added to result image (default is None)

    Returns
    -------
    np.ndarray
        NumPy array including both image and detection box

    """
    ymin, xmin, ymax, xmax = box
    point1, point2 = (int(xmin), int(ymin)), (int(xmax), int(ymax))
    box_color = [random.randint(0, 255) for _ in range(3)]
    line_thickness = round(0.002 * (image.shape[0] + image.shape[1]) / 2) + 1

    cv2.rectangle(img=image, pt1=point1, pt2=point2, color=box_color, thickness=line_thickness, lineType=cv2.LINE_AA)

    if label:
        font_thickness = max(line_thickness - 1, 1)
        font_face = 0
        font_scale = line_thickness / 3
        font_color = (255, 255, 255)
        text_size = cv2.getTextSize(text=label, fontFace=font_face, fontScale=font_scale, thickness=font_thickness)[0]
        # Calculate rectangle coordinates
        rectangle_point1 = point1
        rectangle_point2 = (point1[0] + text_size[0], point1[1] - text_size[1] - 3)
        # Add filled rectangle
        cv2.rectangle(img=image, pt1=rectangle_point1, pt2=rectangle_point2, color=box_color, thickness=-1, lineType=cv2.LINE_AA)
        # Calculate text position
        text_position = point1[0], point1[1] - 3
        # Add text with label to filled rectangle
        cv2.putText(img=image, text=label, org=text_position, fontFace=font_face, fontScale=font_scale, color=font_color, thickness=font_thickness, lineType=cv2.LINE_AA)
    return image

In [ ]:
from typing import Dict

from openvino.runtime.utils.data_helpers import OVDict


def visualize_inference_result(inference_result: OVDict, image: np.ndarray, labels_map: Dict, detections_limit: Optional[int] = None):
    """
    Helper function for visualizing inference result on the image

    Parameters
    ----------
    inference_result : OVDict
        Result of the compiled model inference on the test image
    image : np.ndarray
        Original image to use for visualization
    labels_map : Dict
        Dictionary with mappings of detection classes numbers and its names
    detections_limit : int, optional
        Number of detections to show on the image, if not provided all detections will be shown (default is None)
    """
    detection_boxes: np.ndarray = inference_result.get("detection_boxes")
    detection_classes: np.ndarray = inference_result.get("detection_classes")
    detection_scores: np.ndarray = inference_result.get("detection_scores")
    num_detections: np.ndarray = inference_result.get("num_detections")

    detections_limit = int(
        min(detections_limit, num_detections[0])
        if detections_limit is not None
        else num_detections[0]
    )

    # Normalize detection boxes coordinates to original image size
    original_image_height, original_image_width, _ = image.shape
    normalized_detection_boxex = detection_boxes[::] * [
        original_image_height,
        original_image_width,
        original_image_height,
        original_image_width,
    ]

    image_with_detection_boxex = np.copy(image)

    for i in range(detections_limit):
        detected_class_name = labels_map[int(detection_classes[0, i])]
        score = detection_scores[0, i]
        label = f"{detected_class_name} {score:.2f}"
        add_detection_box(
            box=normalized_detection_boxex[0, i],
            image=image_with_detection_boxex,
            label=label,
        )

    plt.imshow(image_with_detection_boxex)

TensorFlow Object Detection model ([faster_rcnn_resnet50_v1_640x640](https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1)) used in this notebook was trained on [COCO 2017](https://cocodataset.org/) dataset with 91 classes.
For better visualization experience we can use COCO dataset labels with human readable class names instead of class numbers or indexes. 

We can download COCO dataset classes labels from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/):

In [ ]:
coco_labels_file_path = Path("./data/coco_91cl.txt")

download_file(
    url="https://raw.githubusercontent.com/openvinotoolkit/open_model_zoo/master/data/dataset_classes/coco_91cl.txt",
    filename=coco_labels_file_path.name,
    directory=coco_labels_file_path.parent,
)

Then we need to create dictionary `coco_labels_map` with mappings between detection classes numbers and its names from the downloaded file:

In [ ]:
with open(coco_labels_file_path, "r") as file:
    coco_labels = file.read().strip().split("\n")
    coco_labels_map = dict(enumerate(coco_labels, 1))

print(coco_labels_map)

Finally, we are ready to visualize model inference results on the original test image:

In [ ]:
visualize_inference_result(
    inference_result=inference_result,
    image=image,
    labels_map=coco_labels_map,
    detections_limit=5,
)

## Next Steps

This section contains suggestions on how to additionally improve the performance of your application using OpenVINO.

### Async inference pipeline
The key advantage of the Async API is that when a device is busy with inference, the application can perform other tasks in parallel (for example, populating inputs or scheduling other requests) rather than wait for the current inference to complete first. To understand how to perform async inference using openvino, refer to the [Async API tutorial](../115-async-api/115-async-api.ipynb).

### Integration preprocessing to model

Preprocessing API enables making preprocessing a part of the model reducing application code and dependency on additional image processing libraries. 
The main advantage of Preprocessing API is that preprocessing steps will be integrated into the execution graph and will be performed on a selected device (CPU/GPU etc.) rather than always being executed on CPU as part of an application. This will improve selected device utilization.

For more information, refer to the [Optimize Preprocessing tutorial](../118-optimize-preprocessing/118-optimize-preprocessing.ipynb) and to the overview of [Preprocessing API](https://docs.openvino.ai/2023.0/openvino_docs_OV_Runtime_UG_Preprocessing_Overview.html).
